In [78]:
import hashlib
import random
import os

In [147]:
class ConsistentHash():
    def __init__(self, keys_to_hash, nodes, hashes_per_node=1):
        self.keys_to_hash = keys_to_hash
        self.nodes = nodes
        self.hashes_per_node = hashes_per_node
        self.node_hashes = {}
        self.node_tuples = []
        self.hash_assignments = {}
        self.node_assignments = {}
        for node in nodes:
            self.add_node(node)
        for key in keys_to_hash:
            self.add_key(key)
    def get_hash_key(self, key):
        #Gets the hash for a key
        return hashlib.md5(str(key).encode()).hexdigest()
    def get_hash_node(self, node):
        #Gets multiple hashes for a node in a deterministic manner
        return [hashlib.md5((str(node) + str(hash_iter)).encode()).hexdigest() for hash_iter in range(self.hashes_per_node)]
    def assign_to_node(self, key):
        #Assigns a key to the correct node
        key_hash = self.hash_assignments[key]
        self.node_assignments[key] = self.node_tuples[0][1]
        for h, node in self.node_tuples:
            if h > key_hash:
                self.node_assignments[key] = node
                break
    def reassign_keys(self):
        #Reassigns keys when a node has been added or deleted
        self.node_tuples = sorted([x for x in self.node_tuples if x[1] in self.node_hashes], key=lambda x: x[0])
        for key in self.node_assignments: #Won't do anything if no node added yet
            self.assign_to_node(key)        
    def add_key(self, key):
        #Adds a key
        self.hash_assignments[key] = self.get_hash_key(key)
        self.assign_to_node(key)
    def delete_key(self, key):
        #Deletes a key
        del self.hash_assignments[key]
        del self.node_assignments[key]
    def add_node(self, node):
        #Adds a node
        hashes = self.get_hash_node(node)
        self.node_hashes[node] = hashes
        self.node_tuples.extend([(h, node) for h in hashes])
        self.reassign_keys()
    def delete_node(self, node):
        #Deletes a node
        del self.node_hashes[node]
        self.reassign_keys()
    
    
            
    

In [160]:
chash = ConsistentHash(list(range(50)), ["serv1", "serv2", "serv3", "serv4"], 10)

In [161]:
chash.node_assignments

{0: 'serv2',
 1: 'serv1',
 2: 'serv1',
 3: 'serv3',
 4: 'serv4',
 5: 'serv2',
 6: 'serv4',
 7: 'serv2',
 8: 'serv1',
 9: 'serv1',
 10: 'serv2',
 11: 'serv2',
 12: 'serv1',
 13: 'serv1',
 14: 'serv4',
 15: 'serv3',
 16: 'serv1',
 17: 'serv1',
 18: 'serv1',
 19: 'serv4',
 20: 'serv3',
 21: 'serv1',
 22: 'serv4',
 23: 'serv2',
 24: 'serv4',
 25: 'serv2',
 26: 'serv1',
 27: 'serv3',
 28: 'serv2',
 29: 'serv1',
 30: 'serv2',
 31: 'serv4',
 32: 'serv2',
 33: 'serv4',
 34: 'serv2',
 35: 'serv4',
 36: 'serv4',
 37: 'serv4',
 38: 'serv4',
 39: 'serv2',
 40: 'serv2',
 41: 'serv2',
 42: 'serv3',
 43: 'serv4',
 44: 'serv3',
 45: 'serv1',
 46: 'serv2',
 47: 'serv2',
 48: 'serv2',
 49: 'serv3'}

In [162]:
chash.node_hashes

{'serv1': ['4f6dbed3d1014df64b899072f57cb64c',
  '58ca1d95c96e4d774f880cd8771902e9',
  '495be8b0510f521482b85d6834f8be68',
  'c586546ec16168347e7a8b213c513d93',
  '70f6b533bd96558cd9846cb4f640318c',
  'cc46fff9e203d7924725c0f805fdcfc7',
  '4012c9ce9a83469339ebd4d3dab48b11',
  '405cbb2dd96cf1dde52945d5eba8d28f',
  '2b27c01586926470411ce82ff87e0495',
  '787a3522d7966faff6483995cb97cb88'],
 'serv2': ['66af78371493045a6367209758062527',
  '591acb52f23a4dfde8ed6c65a1922ae6',
  'bbb64d5e7f96138218fcc9946814df1a',
  '6839e1cb89b7caf089dc73e4ee1a0464',
  'd7596b495ff1418fb353c7a88a4fce36',
  '38605b55f05d4630e6b160464366c7eb',
  '50354aa17c1bb3aa72714aefed2a84c2',
  '949f722f61be6e2c148b2214c084e8ec',
  'e707513bb73eeb641a7d47527eecff1e',
  'e890fe51fc61c1276a0e826e84510099'],
 'serv3': ['c6cef123ac88f74771d08a41b7b4ed1c',
  '2a25f27b59d1c057e3269db54626a796',
  '104c7de768a21e27a7811f26777b9d55',
  'd79fb4f7e35210cd233de3854fc50773',
  '81e0537bac3fc94d65e33279929d5a59',
  '421f112fa5e3ac32f7

In [163]:
chash.add_key(51)

In [164]:
chash.node_tuples

[('104c7de768a21e27a7811f26777b9d55', 'serv3'),
 ('1244a4c0c5538f2d7f00485fd78fe427', 'serv4'),
 ('28e5c686fef065bdb53b43d8aa344093', 'serv4'),
 ('2a25f27b59d1c057e3269db54626a796', 'serv3'),
 ('2b27c01586926470411ce82ff87e0495', 'serv1'),
 ('310a9994b3799b3b567eb9da1c1ecf4c', 'serv3'),
 ('38605b55f05d4630e6b160464366c7eb', 'serv2'),
 ('39e3319ecfa0a0417752b93ca62c938b', 'serv4'),
 ('4012c9ce9a83469339ebd4d3dab48b11', 'serv1'),
 ('405cbb2dd96cf1dde52945d5eba8d28f', 'serv1'),
 ('421f112fa5e3ac32f77ae35075a8cd07', 'serv3'),
 ('495be8b0510f521482b85d6834f8be68', 'serv1'),
 ('4f6dbed3d1014df64b899072f57cb64c', 'serv1'),
 ('50354aa17c1bb3aa72714aefed2a84c2', 'serv2'),
 ('5756684ff861687bf03c3755dfff080d', 'serv3'),
 ('57bd5c97f8f64d585280826d6d93ccd5', 'serv4'),
 ('58ca1d95c96e4d774f880cd8771902e9', 'serv1'),
 ('591acb52f23a4dfde8ed6c65a1922ae6', 'serv2'),
 ('66af78371493045a6367209758062527', 'serv2'),
 ('67b2b2b4f92c6b17ae6ce2dec379f380', 'serv4'),
 ('6839e1cb89b7caf089dc73e4ee1a0464', 's

In [165]:
chash.delete_node("serv4")

In [166]:
chash.node_assignments

{0: 'serv2',
 1: 'serv1',
 2: 'serv1',
 3: 'serv3',
 4: 'serv2',
 5: 'serv2',
 6: 'serv3',
 7: 'serv2',
 8: 'serv1',
 9: 'serv1',
 10: 'serv2',
 11: 'serv2',
 12: 'serv1',
 13: 'serv1',
 14: 'serv2',
 15: 'serv3',
 16: 'serv1',
 17: 'serv1',
 18: 'serv1',
 19: 'serv3',
 20: 'serv3',
 21: 'serv1',
 22: 'serv2',
 23: 'serv2',
 24: 'serv3',
 25: 'serv2',
 26: 'serv1',
 27: 'serv3',
 28: 'serv2',
 29: 'serv1',
 30: 'serv2',
 31: 'serv1',
 32: 'serv2',
 33: 'serv3',
 34: 'serv2',
 35: 'serv3',
 36: 'serv3',
 37: 'serv2',
 38: 'serv2',
 39: 'serv2',
 40: 'serv2',
 41: 'serv2',
 42: 'serv3',
 43: 'serv3',
 44: 'serv3',
 45: 'serv1',
 46: 'serv2',
 47: 'serv2',
 48: 'serv2',
 49: 'serv3',
 51: 'serv3'}

In [167]:
chash.add_node("serv4")

In [168]:
chash.node_assignments

{0: 'serv2',
 1: 'serv1',
 2: 'serv1',
 3: 'serv3',
 4: 'serv4',
 5: 'serv2',
 6: 'serv4',
 7: 'serv2',
 8: 'serv1',
 9: 'serv1',
 10: 'serv2',
 11: 'serv2',
 12: 'serv1',
 13: 'serv1',
 14: 'serv4',
 15: 'serv3',
 16: 'serv1',
 17: 'serv1',
 18: 'serv1',
 19: 'serv4',
 20: 'serv3',
 21: 'serv1',
 22: 'serv4',
 23: 'serv2',
 24: 'serv4',
 25: 'serv2',
 26: 'serv1',
 27: 'serv3',
 28: 'serv2',
 29: 'serv1',
 30: 'serv2',
 31: 'serv4',
 32: 'serv2',
 33: 'serv4',
 34: 'serv2',
 35: 'serv4',
 36: 'serv4',
 37: 'serv4',
 38: 'serv4',
 39: 'serv2',
 40: 'serv2',
 41: 'serv2',
 42: 'serv3',
 43: 'serv4',
 44: 'serv3',
 45: 'serv1',
 46: 'serv2',
 47: 'serv2',
 48: 'serv2',
 49: 'serv3',
 51: 'serv4'}

In [169]:
chash.delete_key(37)

In [170]:
chash.node_assignments

{0: 'serv2',
 1: 'serv1',
 2: 'serv1',
 3: 'serv3',
 4: 'serv4',
 5: 'serv2',
 6: 'serv4',
 7: 'serv2',
 8: 'serv1',
 9: 'serv1',
 10: 'serv2',
 11: 'serv2',
 12: 'serv1',
 13: 'serv1',
 14: 'serv4',
 15: 'serv3',
 16: 'serv1',
 17: 'serv1',
 18: 'serv1',
 19: 'serv4',
 20: 'serv3',
 21: 'serv1',
 22: 'serv4',
 23: 'serv2',
 24: 'serv4',
 25: 'serv2',
 26: 'serv1',
 27: 'serv3',
 28: 'serv2',
 29: 'serv1',
 30: 'serv2',
 31: 'serv4',
 32: 'serv2',
 33: 'serv4',
 34: 'serv2',
 35: 'serv4',
 36: 'serv4',
 38: 'serv4',
 39: 'serv2',
 40: 'serv2',
 41: 'serv2',
 42: 'serv3',
 43: 'serv4',
 44: 'serv3',
 45: 'serv1',
 46: 'serv2',
 47: 'serv2',
 48: 'serv2',
 49: 'serv3',
 51: 'serv4'}

In [171]:
chash.add_key(37)
chash.node_assignments

{0: 'serv2',
 1: 'serv1',
 2: 'serv1',
 3: 'serv3',
 4: 'serv4',
 5: 'serv2',
 6: 'serv4',
 7: 'serv2',
 8: 'serv1',
 9: 'serv1',
 10: 'serv2',
 11: 'serv2',
 12: 'serv1',
 13: 'serv1',
 14: 'serv4',
 15: 'serv3',
 16: 'serv1',
 17: 'serv1',
 18: 'serv1',
 19: 'serv4',
 20: 'serv3',
 21: 'serv1',
 22: 'serv4',
 23: 'serv2',
 24: 'serv4',
 25: 'serv2',
 26: 'serv1',
 27: 'serv3',
 28: 'serv2',
 29: 'serv1',
 30: 'serv2',
 31: 'serv4',
 32: 'serv2',
 33: 'serv4',
 34: 'serv2',
 35: 'serv4',
 36: 'serv4',
 38: 'serv4',
 39: 'serv2',
 40: 'serv2',
 41: 'serv2',
 42: 'serv3',
 43: 'serv4',
 44: 'serv3',
 45: 'serv1',
 46: 'serv2',
 47: 'serv2',
 48: 'serv2',
 49: 'serv3',
 51: 'serv4',
 37: 'serv4'}